In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns
import scipy

sns.set_context('notebook', font_scale=1.5)

In [2]:
import warnings
warnings.simplefilter('ignore', FutureWarning)

**1**. (25 points)

In this exercise, we will practice using Pandas dataframes to explore and summarize a data set `heart`.

This data contains the survival time after receiving a heart transplant, the age of the patient and whether or not the survival time was censored

- Number of Observations - 69
- Number of Variables - 3

Variable name definitions::

- survival - Days after surgery until death
- censors - indicates if an observation is censored. 1 is uncensored
- age - age at the time of surgery

Answer the following questions (5 points each) with respect to the `heart` data set:

- How many patients were censored?
- What is the correlation coefficient between age and survival for uncensored patients? 
- What is the average age for censored and uncensored patients?
- What is the average survival time for censored and uncensored patients under the age of 45?
- What is the survival time of the youngest and oldest uncensored patient?


In [3]:
import statsmodels.api as sm
heart = sm.datasets.heart.load_pandas().data
heart.head(n=6)

,survival,censors,age
0,15.0,1.0,54.3
1,3.0,1.0,40.4
2,624.0,1.0,51.0
3,46.0,1.0,42.5
4,127.0,1.0,48.0
5,64.0,1.0,54.6


In [4]:
#How many patients were censored?
heart[heart.censors==0].shape[0]

24

24 patients were censored.

In [5]:
#What is the correlation coefficient between age and survival for uncensored patients?
uncensored = heart[heart.censors==1]
scipy.stats.pearsonr(uncensored.age, uncensored.survival)[0]

0.0032564992832119144

The correlation coefficient between age and survival for uncensored patients is 0.003.

In [6]:
#What is the average age for censored and uncensored patients?
heart.groupby("censors").agg({'age':"mean"})

,age
censors,
0.0,41.729167
1.0,48.484444


The average age for censored patients is 48.48, for uncensored patients is 41.73.

In [7]:
#What is the average survival time for censored and uncensored patients under the age of 45?
heart[heart.age<45].groupby("censors").agg({"survival":"mean"})

,survival
censors,
0.0,712.818182
1.0,169.909091


The average survival time for censored patients under 45 years old is 169.9 days.
The average survival time for uncensored patients under 45 years old is 712.8 days.

In [8]:
#What is the survival time of the youngest and oldest uncensored patient?
survival_min = uncensored[uncensored.age == uncensored.age.min()].survival
survival_max = uncensored[uncensored.age == uncensored.age.max()].survival

In [9]:
survival_min

41    228.0
Name: survival, dtype: float64

The survival time of the yougest uncensored patient is 228.

In [10]:
survival_max

17    60.0
Name: survival, dtype: float64

The survival time of the oldest uncensored patient is 60.

**2**. (35 points)

- Consider a sequence of $n$ Bernoulli trials with success probabilty $p$ per trial. A string of consecutive successes is known as a success *run*. Write a function that returns the counts for runs of length $k$ for each $k$ observed in a dictionary. (10 points)

For example: if the trials were [0, 1, 0, 1, 1, 0, 0, 0, 0, 1], the function should return 
```
{1: 2, 2: 1})
```

Test that it does so.

- What is the probability of observing at least one run of length 5 or more when $n=100$ and $p=0.5$?. Estimate this from 100,000 simulated experiments. Is this more, less or equally likely than finding runs of length 7 or more when $p=0.7$? (10 points)

- There is an exact solution

$$
s_n = \sum_{i=1}^n{f_i} \\
f_n = u_n - \sum_{i=1}^{n-1} {f_i u_{n-i}} \\
u_n = p^k - \sum_{i=1}^{k-1} u_{n-i} p^i
$$

Implement the exact solution using caching to avoid re-calculations and calculate the same two probabilities found by simulation. (15 points)

In [11]:
def len_run(str_list):
    s_str = "".join([str(s) for s in str_list]).split("0")
    run = [s for s in s_str if s != ""]
    unique = set(run)
    l = [len(u) for u in unique]
    count = []
    for e in unique:
        count.append(run.count(e))
    
    return dict(zip(l, count))

In [12]:
s = [0, 1, 0, 1, 1, 0, 0, 0, 0, 1]
len_run(s)

{1: 2, 2: 1}

In [13]:
n = 100
p = 0.5
N = 100000
result = []
for i in range(N):
    s = [np.random.binomial(1,p) for j in range(n)]
    run_len = len_run(s).keys()
    whether_5 = np.sum([e >= 5 for e in run_len])
    if whether_5>=1:
        result.append(1)
    else:
        result.append(0)

In [14]:
np.mean(result)

0.81115999999999999

In [15]:
n = 100
p = 0.7
N = 100000
result = []
for i in range(N):
    s = [np.random.binomial(1,p) for j in range(n)]
    run_len = len_run(s).keys()
    whether_5 = np.sum([e >= 7 for e in run_len])
    if whether_5>=1:
        result.append(1)
    else:
        result.append(0)

In [16]:
np.mean(result)

0.94891999999999999

In [17]:
from functools import lru_cache

@lru_cache()
def u(n,k,p):
    if n < k:
        return 0
    else:
        return p**k - sum(u(n-i, k, p)*p**i for i in range(1, k))

@lru_cache()
def f(n,k,p):
    return u(n,k,p) - sum(f(i,k,p)*u(n-i,k,p) for i in range(1,n))

@lru_cache()
def s(n,k,p):
    return sum(f(i,k,p) for i in range(1, n+1))

In [18]:
s(100,5,0.5)

0.8101095991963579

In [19]:
s(100,7,0.7)

0.9491817984156692

**3**. (40 points)

Given matrix $M$
```python
      [[7, 8, 8],
       [1, 3, 8],
       [9, 2, 1]]
```

- Normalize the given matrix $M$ so that all rows sum to 1.0.  (5 points)
- The normalized matrix can then be considered as a transition matrix $P$ for a Markov chain. Find the stationary distribution of this matrix in the following ways using `numpy` and `numpy.linalg` (or `scipy.linalg`):
    - By repeated matrix multiplication of a random probability vector $v$ (a row vector normalized to sum to 1.0) with $P$ using matrix multiplication with `np.dot`. (5 points)
    - By raising the matrix $P$ to some large power until it doesn't change with higher powers (see `np.linalg.matrix_power`) and then calculating $vP$ (10 points)
    - From the equation for stationarity $wP = w$, we can see that $w$ must be a left eigenvector of $P$ with eigenvalue $1$ (Note: np.linalg.eig returns the right eigenvectors, but the left eigenvector of a matrix is the right eigenvector of the transposed matrix). Use this to find $w$ using `np.linalg.eig`. (20 points)

Suppose $w = (w_1, w_2, w_3)$. Then from $wP = w$, we have:
\begin{align}
w_1 P_{11} + w_2 P_{21} + w_3 P_{31} &= w_1 \\
w_1 P_{12} + w_2 P_{22} + w_3 P_{32} &= w_2 \\
w_1 P_{13} + w_2 P_{23} + w_3 P_{331} &= w_3 \\
\end{align}
This is a singular system, but we also know that $w_1 + w_2 + w_3 = 1$. Use these facts to set up a linear system of equations that can be solved with `np.linalg.solve` to find $w$.

In [20]:
M = np.array([[7,8,8],[1,3,8],[9,2,1]])
M

array([[7, 8, 8],
       [1, 3, 8],
       [9, 2, 1]])

In [21]:
sum_row = M.sum(axis=1)
normalized_M = M/sum_row[:,None]
normalized_M

array([[ 0.30434783,  0.34782609,  0.34782609],
       [ 0.08333333,  0.25      ,  0.66666667],
       [ 0.75      ,  0.16666667,  0.08333333]])

In [38]:
P = normalized_M
v = np.array([0.2, 0.7, 0.1])

In [28]:
v_old = v
v_new = np.dot(v_old,P)
while any(v_old != v_new):
    v_old = v_new
    v_new = np.dot(v_old, P)
w1 = v_old
w1

array([ 0.39862184,  0.2605972 ,  0.34078096])

In [36]:
P_s = np.linalg.matrix_power(P,20)
P_s

array([[ 0.39862184,  0.2605972 ,  0.34078096],
       [ 0.39862184,  0.2605972 ,  0.34078096],
       [ 0.39862184,  0.2605972 ,  0.34078096]])

In [39]:
w2 = np.dot(v, P_s)
w2

array([ 0.39862184,  0.2605972 ,  0.34078096])

In [40]:
W,V = np.linalg.eig(P.T)
w3 = V[:, 0]/V[:, 0].sum()
w3

array([ 0.39862184+0.j,  0.26059720+0.j,  0.34078096+0.j])

In [41]:
A = np.r_[P.T-np.eye(3),[np.ones(3)]][1:,:]
b = np.array([0,0,1])
w4 = np.linalg.solve(A,b)
w4

array([ 0.39862184,  0.2605972 ,  0.34078096])